In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset,load_dataset
import ast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import default_data_collator,TrainerCallback,set_seed
import torch
seed = 42
set_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlbse-datasets/NLBSE_Dataset_Java.csv
/kaggle/input/nlbse-datasets/NLBSE_Dataset_Python.csv
/kaggle/input/nlbse-datasets/NLBSE_Dataset_Pharo.csv
Using device: cuda


In [2]:
df = pd.read_csv('/kaggle/input/nlbse-datasets/NLBSE_Dataset_Pharo.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7781 entries, 0 to 7780
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             7781 non-null   object
 1   comment_sentence  7781 non-null   object
 2   labels            7781 non-null   object
 3   types             7781 non-null   object
dtypes: object(4)
memory usage: 243.3+ KB


In [4]:
df.head(10)
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb")
import wandb

# Replace YOUR_API_KEY with your actual API key
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: himel6087 (himel6087-bangladesh-university-of-engineering-and-techn). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df.drop_duplicates(subset=['comment_sentence'], keep='first', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6457 entries, 0 to 7780
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             6457 non-null   object
 1   comment_sentence  6457 non-null   object
 2   labels            6457 non-null   object
 3   types             6457 non-null   object
dtypes: object(4)
memory usage: 252.2+ KB


In [6]:
null_rows = df[df['comment_sentence'].isnull()]

print("Rows with null values in 'comment_sentence':")
print(null_rows)

Rows with null values in 'comment_sentence':
Empty DataFrame
Columns: [class, comment_sentence, labels, types]
Index: []


In [7]:
df_cleaned = df.dropna(subset=['comment_sentence'])
print("DataFrame shape after removing nulls:", df_cleaned.shape)
df_cleaned.info()

DataFrame shape after removing nulls: (6457, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 6457 entries, 0 to 7780
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             6457 non-null   object
 1   comment_sentence  6457 non-null   object
 2   labels            6457 non-null   object
 3   types             6457 non-null   object
dtypes: object(4)
memory usage: 252.2+ KB


In [8]:
pattern = r'https?://\S+|\t'
rows_with_pattern = df_cleaned.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)

# Count rows with patterns
num_rows_with_pattern = rows_with_pattern.sum()
print(f"\nNumber of rows containing patterns: {num_rows_with_pattern}")

# Remove `//` or `*` from all columns
df_cleaned = df_cleaned.replace(pattern, '', regex=True)


Number of rows containing patterns: 9


In [9]:
df = df_cleaned
df['combo'] = df['comment_sentence'] +"  |  "+  df['class']
pharo_dataset = Dataset.from_pandas(df)
# Split the dataset into train and validation subsets
train_test_split = pharo_dataset.train_test_split(test_size=0.2, seed=42)

# Extract train and validation datasets
pharo_train = train_test_split['train']
pharo_test = train_test_split['test']
pharo_labels = ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']

In [ ]:
# Use Hugging Face's default data collator
data_collator = default_data_collator
num_labels = len(pharo_labels)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["combo"], truncation=True, padding="max_length", max_length=128)
tokenized_train = pharo_train.map(tokenize_function, batched=True)
tokenized_test = pharo_test.map(tokenize_function, batched=True)

# Convert labels to tensors
def encode_labels(examples):
    if isinstance(examples['labels'], str):
        examples["labels"]=examples["labels"].replace(" ", ",")
        labels = ast.literal_eval(examples['labels'])
    else:
        labels = examples['labels']
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float32)
    return {'labels': labels}
tokenized_train = tokenized_train.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

# Format datasets for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-large",
    num_labels=num_labels,
    problem_type="multi_label_classification"
).to(device)

# Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = (logits > 0.5).astype(int)  # Multi-label threshold
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    return {"precision": precision, "recall": recall, "f1": f1}


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1.3269664638093366e-05,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=24,
    weight_decay= 0.09130339417806095,
    logging_dir="./logs1",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Use validation loss to select the best model
    greater_is_better=False,  # Lower validation loss is better
)


# Custom callback to monitor F1 score and restart training if F1 is zero
class RestartIfF1ZeroCallback(TrainerCallback):
    def __init__(self, trainer, train_dataset, eval_dataset, tokenizer, data_collator):
        self.trainer = trainer
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.tokenizer = tokenizer
        self.data_collator = data_collator

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        # Check F1 score
        if "eval_f1" in metrics and metrics["eval_f1"] == 0.0:
            print("F1 score is zero. Restarting training...")
            
            # Reinitialize model
            self.trainer.model = AutoModelForSequenceClassification.from_pretrained(
                "roberta-base",
                num_labels=num_labels,
                problem_type="multi_label_classification"
            ).to(device)

            # Restart training
            self.trainer.train_dataset = self.train_dataset
            self.trainer.eval_dataset = self.eval_dataset
            self.trainer.tokenizer = self.tokenizer
            self.trainer.data_collator = self.data_collator

            self.trainer.train()

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Add custom callback
trainer.add_callback(RestartIfF1ZeroCallback(trainer, tokenized_train, tokenized_test, tokenizer, data_collator))

# Train the model
trainer.train()

# Save the model
trainer.save_model("./roberta-pharo-multi-label")
tokenizer.save_pretrained("./roberta-pharo-multi-label")

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

Map:   0%|          | 0/5165 [00:00<?, ? examples/s]

Map:   0%|          | 0/1292 [00:00<?, ? examples/s]

Map:   0%|          | 0/5165 [00:00<?, ? examples/s]

Map:   0%|          | 0/1292 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_13377/1096639213.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
0,No log,0.232258,0.857923,0.532203,0.656904
1,0.324300,0.130035,0.935802,0.770847,0.845353
2,0.160400,0.081535,0.954579,0.869153,0.909865
3,0.088700,0.064269,0.969985,0.898305,0.932770
4,0.056200,0.047081,0.975542,0.946441,0.960771
5,0.056200,0.036072,0.983110,0.947119,0.964779
6,0.036100,0.031638,0.984061,0.962712,0.973269
7,0.021900,0.025095,0.982970,0.978305,0.980632
8,0.015700,0.024521,0.982216,0.973559,0.977869
9,0.010400,0.023612,0.982947,0.976949,0.979939


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Evaluation Results: {'eval_loss': 0.017641184851527214, 'eval_precision': 0.9884510869565217, 'eval_recall': 0.9864406779661017, 'eval_f1': 0.987444859178826, 'eval_runtime': 29.8517, 'eval_samples_per_second': 43.281, 'eval_steps_per_second': 2.713, 'epoch': 15.944272445820433}


In [13]:
import numpy as np
import pandas as pd
import time
import torch

def evaluate_roberta(new_dataset, model, tokenizer, labels, batch_size=16, device='cuda'):
    """
    Evaluate a fine-tuned RoBERTa-large model on a new dataset.

    Args:
        new_dataset (dict): A dictionary with 'combo' (input texts) and 'labels' (one-hot encoded labels).
        model (transformers.PreTrainedModel): A trained RoBERTa model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer corresponding to the model.
        labels (list): List of label categories.
        batch_size (int): Batch size for evaluation.
        device (str): Device to use ('cuda' or 'cpu').

    Returns:
        pd.DataFrame: DataFrame containing precision, recall, and F1-score for each label.
        float: Average runtime per batch.
    """
    # Move model to device
    model.to(device)

    # Prepare data
    texts = new_dataset['combo']
    true_labels = np.array(new_dataset['labels'])

    # Tokenize the inputs
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Split into batches
    num_batches = (len(texts) + batch_size - 1) // batch_size
    predictions = []
    start_time = time.time()

    # Perform inference in batches
    for i in range(num_batches):
        batch_input_ids = input_ids[i * batch_size: (i + 1) * batch_size]
        batch_attention_mask = attention_mask[i * batch_size: (i + 1) * batch_size]

        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = (logits.sigmoid() > 0.5).int().cpu().numpy()
            predictions.append(preds)
            
    end_time = time.time()
    avg_runtime = (end_time - start_time) / num_batches

    # Concatenate predictions
    predictions = np.vstack(predictions)

    # Evaluate metrics for each label
    metrics = []
    for i, label in enumerate(labels):
        tp = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 1))
        fp = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 1))
        fn = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 0))
        tn = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 0))

        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics.append({'label': label, 'precision': precision, 'recall': recall, 'f1': f1})

    # Convert metrics to DataFrame
    metrics_df = pd.DataFrame(metrics)
    return metrics_df, avg_runtime

In [17]:
import re
# Use the model and tokenizer from the training session
trained_model = model  # Replace with your trained model instance
trained_tokenizer = tokenizer  # Replace with your tokenizer instance
test = load_dataset('NLBSE/nlbse25-code-comment-classification')['pharo_test']
# def clean_text(example):
#     if "combo" in example:  # Replace "text" with the relevant column name
#         example["combo"] = re.sub(r'https?://\S+|\t', '', example["combo"], flags=re.MULTILINE)
#         example["combo"] = example["combo"].strip()  # Remove leading/trailing spaces
#     return example
# test = test.map(clean_text)
# Define label names
labels =  ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']

# Evaluate the model
metrics, avg_runtime = evaluate_roberta(test, trained_model, trained_tokenizer, labels)

print("Evaluation Metrics:")
print(metrics)
print(f"Average runtime per batch: {avg_runtime:.4f} seconds")
from huggingface_hub import login

# Retrieve API token from Kaggle secrets
# import os
# huggingface_token = user_secrets.get_secret("LLama Token")

# login(token=huggingface_token)
# trained_model.push_to_hub("NLBSE-Pharo-final")
# trained_tokenizer.push_to_hub("NLBSE-Pharo-final")

Evaluation Metrics:
                     label  precision    recall        f1
0  Keyimplementationpoints   0.733333  0.511628  0.602740
1                  Example   0.921739  0.890756  0.905983
2         Responsibilities   0.585714  0.788462  0.672131
3          Classreferences   0.500000  0.750000  0.600000
4                   Intent   0.843750  0.900000  0.870968
5              Keymessages   0.739130  0.790698  0.764045
6            Collaborators   0.454545  0.500000  0.476190
Average runtime per batch: 0.2286 seconds
